In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats # For beta calculation (linear regression)

# --- Configuration ---
# IMPORTANT: Adjust these paths and names to match your Excel file
excel_file_name = 'Market Data.xlsx'
sheet_name = 'Sheet1' # Change this if your data is on a different sheet

# Define stock ticker columns (ensure these match your Excel column headers exactly)
stock_price_cols = ['MSFT', 'AAPL', 'JPM', 'TSLA']
# Define the market index column (ensure this matches your Excel column header exactly)
market_index_col = 'S&P 500'
# Define the Risk-Free Rate column name
risk_free_rate_col = 'Risk-Free Rate'

# --- Step 1: Load Data from Excel ---
print("--- Step 1: Loading Data from Excel ---")
try:
    # Load the data, setting 'date' column as the index and parsing dates
    df = pd.read_excel(excel_file_name, sheet_name=sheet_name, index_col='date', parse_dates=True)

    # Ensure all required columns are present after loading
    required_cols = stock_price_cols + [market_index_col, risk_free_rate_col]
    if not all(col in df.columns for col in required_cols):
        missing_cols = [col for col in required_cols if col not in df.columns]
        raise ValueError(f"Missing required columns in the Excel file: {missing_cols}. "
                         f"Please check your Excel headers and ensure all are present.")

    # Extract the Risk-Free Rate from the loaded DataFrame
    # Assuming the risk-free rate is constant or you want the first value
    risk_free_rate = df[risk_free_rate_col].iloc[0]

    print(f"Data loaded successfully from '{excel_file_name}' - Sheet: '{sheet_name}'.")
    print(f"Extracted Risk-Free Rate: {risk_free_rate}")
    print("First 5 rows of the loaded data:")
    print(df.head())
    print("\n")

except FileNotFoundError:
    print(f"Error: The file '{excel_file_name}' was not found. Please ensure it's in the same directory as the script or provide the full path.")
    exit()
except Exception as e:
    print(f"An error occurred while loading the Excel file: {e}")
    exit()

# --- Step 2: Calculate Daily Returns for Stocks and Market ---
# Returns are calculated as (Current Price / Previous Price) - 1 for each asset.
print("--- Step 2: Calculating Daily Returns ---")
returns_df = pd.DataFrame(index=df.index) # Initialize DataFrame for returns

# Calculate returns for each stock
for col in stock_price_cols:
    returns_df[f'{col}_Return'] = df[col].pct_change()

# Calculate returns for the market index
returns_df[f'{market_index_col}_Return'] = df[market_index_col].pct_change()

# Drop the first row which will contain NaN values due to pct_change() operation
returns_df.dropna(inplace=True)

print("Daily returns calculated for stocks and market index. First 5 rows of returns:")
print(returns_df.head())
print("\n")

# --- Step 3: Calculate Betas for Stocks ---
# Beta measures the sensitivity of a stock's return to the overall market's returns.
# It's calculated using the slope of a linear regression where stock returns are dependent
# on market returns (Beta = Cov(Asset_Return, Market_Return) / Var(Market_Return)).
print("--- Step 3: Calculating Betas ---")
betas = {}
market_returns = returns_df[f'{market_index_col}_Return']

for stock in stock_price_cols:
    stock_returns = returns_df[f'{stock}_Return']

    # Ensure there's enough non-NaN data points for regression
    # filter out NaNs that might occur if returns_df still has some for specific stocks
    common_indices = stock_returns.dropna().index.intersection(market_returns.dropna().index)
    if len(common_indices) > 1: # Need at least 2 points for linregress
        # Perform linear regression using scipy.stats.linregress
        slope, intercept, r_value, p_value, std_err = stats.linregress(
            market_returns.loc[common_indices], stock_returns.loc[common_indices]
        )
        betas[stock] = slope
        print(f"Beta for {stock}: {slope:.4f}")
    else:
        betas[stock] = np.nan # Not enough valid data to calculate beta
        print(f"Not enough data to calculate Beta for {stock}.")
print("\n")

# --- Step 4: Calculate CAPM Expected Returns ---
# The Capital Asset Pricing Model (CAPM) calculates the expected return of an asset
# based on its systematic risk (beta), the risk-free rate, and the market's expected return.
# E(Ri) = Rf + Beta_i * (E(Rm) - Rf)
print("--- Step 4: Calculating CAPM Expected Returns ---")
expected_market_return = market_returns.mean() # Average historical market return as E(Rm)
market_risk_premium = expected_market_return - risk_free_rate

capm_expected_returns = {}
for stock, beta in betas.items():
    if not np.isnan(beta): # Only calculate if beta is available
        e_ri = risk_free_rate + beta * market_risk_premium
        capm_expected_returns[stock] = e_ri
        print(f"Expected Return for {stock} (CAPM): {e_ri:.4%}")
    else:
        capm_expected_returns[stock] = np.nan # Mark as NaN if beta is missing
        print(f"Skipping CAPM Expected Return for {stock} due to missing beta.")
print("\n")

# --- Step 5: Calculate Average Historical Returns ---
# This is the simple average of the daily returns for each stock over the period.
# It's a backward-looking estimate of expected return.
print("--- Step 5: Calculating Average Historical Returns ---")
average_historical_returns = returns_df[[f'{col}_Return' for col in stock_price_cols]].mean()
print(average_historical_returns.apply(lambda x: f"{x:.4%}"))
print("\n")

# --- Step 6: Calculate Individual Stock Volatility (Standard Deviation of Returns) ---
# Standard deviation measures the dispersion of returns around their mean, indicating total risk.
print("--- Step 6: Calculating Individual Stock Volatility (Standard Deviation of Returns) ---")
std_devs = returns_df[[f'{col}_Return' for col in stock_price_cols]].std()
print(std_devs.apply(lambda x: f"{x:.4%}"))
print("\n")

# --- Step 7: Calculate Covariance Matrix ---
# The covariance matrix shows how each pair of stock returns moves together.
# A positive covariance indicates that two assets tend to move in the same direction.
print("--- Step 7: Calculating Covariance Matrix ---")
covariance_matrix = returns_df[[f'{col}_Return' for col in stock_price_cols]].cov()
print(covariance_matrix)
print("\n")

# --- Step 8: Calculate Correlation Matrix ---
# The correlation matrix is a standardized version of the covariance matrix, with values ranging
# from -1 (perfect inverse correlation) to +1 (perfect positive correlation). It helps understand
# diversification benefits.
print("--- Step 8: Calculating Correlation Matrix ---")
correlation_matrix = returns_df[[f'{col}_Return' for col in stock_price_cols]].corr()
print(correlation_matrix)
print("\n")

# --- Step 9: Calculate Sortino Ratio ---
# The Sortino Ratio is a risk-adjusted performance measure similar to the Sharpe Ratio,
# but it only penalizes downside volatility (returns below a Minimum Acceptable Return - MAR).
print("--- Step 9: Calculating Sortino Ratio ---")

# Define MAR (Minimum Acceptable Return) for Sortino Ratio.
# Using the extracted risk_free_rate as MAR, which is common.
mar = risk_free_rate

sortino_ratios = {}
for stock in stock_price_cols:
    returns_col_name = f'{stock}_Return'
    returns = returns_df[returns_col_name]

    # Filter for returns that fall below the MAR
    downside_returns = returns[returns < mar]

    if not downside_returns.empty:
        # Calculate downside deviation: standard deviation of these downside returns
        # This approach calculates the standard deviation of only the returns below the MAR.
        downside_deviation = downside_returns.std()

        if downside_deviation != 0: # Avoid division by zero
            # Get the average return for the stock
            avg_return = returns.mean()
            sortino_ratio = (avg_return - mar) / downside_deviation
            sortino_ratios[stock] = sortino_ratio
        else:
            # Downside deviation is zero (all returns are >= MAR or there's no variance in downside)
            sortino_ratios[stock] = np.nan
    else:
        # No returns fell below the MAR, implying excellent performance above the threshold
        sortino_ratios[stock] = np.inf

# Print Sortino Ratios
for stock, ratio in sortino_ratios.items():
    if ratio == np.inf:
        print(f"Sortino Ratio for {stock}: inf (no returns fell below the MAR)")
    elif np.isnan(ratio):
        print(f"Sortino Ratio for {stock}: NaN (downside deviation is zero)")
    else:
        print(f"Sortino Ratio for {stock}: {ratio:.4f}")

--- Step 1: Loading Data from Excel ---
Data loaded successfully from 'Market Data.xlsx' - Sheet: 'Sheet1'.
Extracted Risk-Free Rate: 0.0025
First 5 rows of the loaded data:
               MSFT    AAPL     JPM    TSLA  S&P 500  Risk-Free Rate
date                                                                
2018-09-04  104.419  54.210  94.817  19.263   2654.8          0.0025
2018-09-05  101.409  53.856  94.355  18.716   2734.6          0.0025
2018-09-06  101.643  52.962  93.952  18.730   2813.4          0.0025
2018-09-07  101.147  52.534  94.133  17.549   2924.6          0.0025
2018-09-10  102.241  51.829  93.631  19.033   2740.4          0.0025


--- Step 2: Calculating Daily Returns ---
Daily returns calculated for stocks and market index. First 5 rows of returns:
            MSFT_Return  AAPL_Return  JPM_Return  TSLA_Return  S&P 500_Return
date                                                                         
2018-09-05    -0.028826    -0.006530   -0.004873    -0.028396   